In [2]:
import json

In [3]:
f = open('mpqa_csds.json', 'r', encoding='utf-8')

In [4]:
data = json.load(f)

In [114]:
limit = 100
count = 0
for obj in data:
    if count > limit:
        break
    s1 = set(['w_head_span', 'w_text', 'clean_head', 'head_start', 'head_end'])
    if s1.issubset(obj.keys()):
        clean, w_head, w_end = assemble_clean_text(obj['w_head_span'], obj['w_text'])
        # print(obj['clean_text'])
        print('ORIGINAL:', obj['clean_head'])
        print('     NEW:', clean[w_head:w_end])
        # print('original:', obj['head_start'], obj['head_end'], 'new:', w_head, w_end)
    count+=1

ORIGINAL: branded as "evil "
     NEW: branded as "evil"
ORIGINAL: "nuclear club "
     NEW: "nuclear club"
ORIGINAL: may have erred
     NEW: may have erred
ORIGINAL: more clearly
     NEW: more clearly
ORIGINAL: real
     NEW: real
ORIGINAL: as far as human rights are concerned
     NEW: as far as human rights are concerned
ORIGINAL: easily qualify as repressive and violating human rights
     NEW: easily qualify as repressive and violating human rights
ORIGINAL: oppressive
     NEW: oppressive
ORIGINAL: not in itself
     NEW: not in itself
ORIGINAL: aggressively
     NEW: aggressively
ORIGINAL: evildoer
     NEW: evildoer
ORIGINAL: therefore
     NEW: therefore
ORIGINAL: scores of
     NEW: scores of
ORIGINAL: seemingly
     NEW: seemingly
ORIGINAL: Perhaps
     NEW: erhaps
ORIGINAL: In other words
     NEW: n other words
ORIGINAL: aggressively
     NEW: 
ORIGINAL: grave human rights violators
     NEW: grave human rights violators
ORIGINAL: irrespective
     NEW: irrespective
ORIG

What to do with possesives? For example: PRESIDENT GEORGE Bush 's National Security Adviser Condoleezza Rice has recently "defined" the context and scope of the term "axis of evil" that Bush used in his State of the Union address a few weeks ago to describe Iraq, Iran and North Korea.

In [1]:
raw_str = "All other countries which are seemingly \nrefraining from acquiring such weapons are doing so not so much \nto escape being branded as \"evil\" but rather because they have \nneither the knowledge nor the money to join the \"nuclear club\"."
clean_str = "All other countries which are seemingly refraining from acquiring such weapons are doing so not so much to escape being branded as \"evil \"but rather because they have neither the knowledge nor the money to join the \"nuclear club \"."

In [82]:
w_text = ["All",
                "other",
                "countries",
                "which",
                "are",
                "seemingly",
                "refraining",
                "from",
                "acquiring",
                "such",
                "weapons",
                "are",
                "doing",
                "so",
                "not",
                "so",
                "much",
                "to",
                "escape",
                "being",
                "branded",
                "as",
                "\"",
                "evil",
                "\"",
                "but",
                "rather",
                "because",
                "they",
                "have",
                "neither",
                "the",
                "knowledge",
                "nor",
                "the",
                "money",
                "to",
                "join",
                "the",
                "\"",
                "nuclear",
                "club",
                "\"",
                "."
            ]

In [9]:
import re

def replace_punc(clean_text, offset):
    s = clean_text
    s, off1 = re.subn(' \.', '.', s)
    s, off2 = re.subn(' ,', ',', s)
    s, off3 = re.subn(' ;', ';', s)
    s, off4 = re.subn(' :', ':', s) 
    offset -= (off1+off2+off3+off4)
    return s, offset

In [49]:
clean, ind = assemble_clean_text_helper(20, 25 ,w_text)
print(clean, ind)

 branded as "evil" 18


In [106]:
#new function
def assemble_clean_text_helper(sentinel, w_text):
    index = 0
    offset = 0
    clean_text = ''
    in_quote = False
    while index < sentinel:
        # print(w_text[index], offset)
        if (index == 0):
            clean_text += w_text[index]
            offset += len(w_text[index])
            index+=1
            continue
        if w_text[index] == '"' or w_text[index] == '\'':
            offset += 1
            if in_quote:
                clean_text += w_text[index]
                in_quote = False
            else:
                clean_text += ' ' + w_text[index] + w_text[index+1]
                index+=1
                # print(w_text[index])
                in_quote = True
        else:
            clean_text += ' '
            offset += 1
            clean_text += w_text[index]    
        offset += len(w_text[index])
        index += 1
        clean_text, offset = replace_punc(clean_text, offset)
    return clean_text, offset 

def assemble_clean_text(w_head_span, w_text):
    w_head_start, w_head_end = w_head_span 
    clean_text = ''
    clean_text, w_head_start_offset = assemble_clean_text_helper(w_head_start, w_text)
    clean_text, w_head_end_offset = assemble_clean_text_helper(w_head_end, w_text)
    clean_text, _ = assemble_clean_text_helper(len(w_text),w_text)
    #add one for space
    return clean_text, w_head_start_offset+1, w_head_end_offset

In [108]:
new_clean, ind = assemble_clean_text_helper(22, w_text)
print(new_clean, ind)

PRESIDENT GEORGE Bush 's National Security Adviser Condoleezza Rice has recently "defined" the context and scope of the term "axis 129


In [100]:
new_clean[124:137]

' "axis of evi'

In [96]:
new_clean, ind = assemble_clean_text_helper(25, w_text)
print(new_clean, ind)

PRESIDENT GEORGE Bush 's National Security Adviser Condoleezza Rice has recently "defined" the context and scope of the term "axis of evil 137


In [84]:
w_text[22]

'"'

In [81]:
clean, w_head, w_end = assemble_clean_text([22, 25], w_text)
print(clean, w_head, w_end)
print(clean[w_head:w_end])

PRESIDENT 0
GEORGE 9
Bush 16
's 21
National 24
Security 33
Adviser 42
Condoleezza 50
Rice 62
has 67
recently 71
" 80
defined
" 88
the 90
context 94
and 102
scope 106
of 112
the 115
term 119
" 124
axis
PRESIDENT 0
GEORGE 9
Bush 16
's 21
National 24
Security 33
Adviser 42
Condoleezza 50
Rice 62
has 67
recently 71
" 80
defined
" 88
the 90
context 94
and 102
scope 106
of 112
the 115
term 119
" 124
axis
of 129
evil 132
PRESIDENT 0
GEORGE 9
Bush 16
's 21
National 24
Security 33
Adviser 42
Condoleezza 50
Rice 62
has 67
recently 71
" 80
defined
" 88
the 90
context 94
and 102
scope 106
of 112
the 115
term 119
" 124
axis
of 129
evil 132
" 137
that 139
Bush 144
used 149
in 154
his 157
State 161
of 167
the 170
Union 174
address 180
a 188
few 190
weeks 194
ago 200
to 204
describe 207
Iraq 216
, 221
Iran 222
and 227
North 231
Korea 237
. 243
PRESIDENT GEORGE Bush 's National Security Adviser Condoleezza Rice has recently "defined" the context and scope of the term "axis of evil" that Bush used in hi

In [89]:
w_text = [
                "PRESIDENT",
                "GEORGE",
                "Bush",
                "'s",
                "National",
                "Security",
                "Adviser",
                "Condoleezza",
                "Rice",
                "has",
                "recently",
                "\"",
                "defined",
                "\"",
                "the",
                "context",
                "and",
                "scope",
                "of",
                "the",
                "term",
                "\"",
                "axis",
                "of",
                "evil",
                "\"",
                "that",
                "Bush",
                "used",
                "in",
                "his",
                "State",
                "of",
                "the",
                "Union",
                "address",
                "a",
                "few",
                "weeks",
                "ago",
                "to",
                "describe",
                "Iraq",
                ",",
                "Iran",
                "and",
                "North",
                "Korea",
                "."
            ]

In [83]:
clean, w_head, w_end = assemble_clean_text([20, 25], w_text)
print(clean, w_head, w_end)

All 0
other 3
countries 9
which 19
are 25
seemingly 29
refraining 39
from 50
acquiring 55
such 65
weapons 70
are 78
doing 82
so 88
not 91
so 95
much 98
to 103
escape 106
being 113
All 0
other 3
countries 9
which 19
are 25
seemingly 29
refraining 39
from 50
acquiring 55
such 65
weapons 70
are 78
doing 82
so 88
not 91
so 95
much 98
to 103
escape 106
being 113
branded 119
as 127
" 130
evil
" 135
All 0
other 3
countries 9
which 19
are 25
seemingly 29
refraining 39
from 50
acquiring 55
such 65
weapons 70
are 78
doing 82
so 88
not 91
so 95
much 98
to 103
escape 106
being 113
branded 119
as 127
" 130
evil
" 135
but 137
rather 141
because 148
they 156
have 161
neither 166
the 174
knowledge 178
nor 188
the 192
money 196
to 202
join 205
the 210
" 214
nuclear
club 222
" 227
. 229
All other countries which are seemingly refraining from acquiring such weapons are doing so not so much to escape being branded as "evil" but rather because they have neither the knowledge nor the money to join the "nucl

In [55]:
clean[w_head:w_end]

'branded as "evil"'

In [ ]:
raw_str2 = "One can name more than a dozen countries which are \ncontemplating the acquisition of mass destruction weapons at one \nspeed or another."
clean_str2 = "One can name more than a dozen countries which are contemplating the acquisition of mass destruction weapons at one speed or another."

## Previous work

In [ ]:
def assemble_clean_text_helper(index, sentinel, offset, clean_text, w_text):
    in_single_quotation = False
    in_double_quotation = False
    while index < sentinel:
        if (index == 0):
            clean_text += w_text[index]
            continue
        if w_text[index] == '"':
            if in_double_quotation:
                in_double_quotation = False
            else:
                clean_text += ' '
                in_double_quotation = True
        else if w_text[index] == '\'':
            if in_single_quotation:
                in_single_quotation = False
            else:
                clean_text += ' '
                in_single_quotation = True
        else if w_text[index] != '.':
            clean_text += ' '
            offset += 1
        clean_text += w_text[index]    
        offset += len(w_text[index])
    return clean_text, offset 

In [ ]:
def assemble_clean_text(w_head_span, w_text):
    w_head_start, w_head_end = w_head_span 
    offset = 0
    index = 0
    clean_text = ''
    clean_text, w_head_start_offset = assemble_clean_text_helper(index, w_head_start, clean_text, w_text)
    clean_text, w_head_end_offset = assemble_clean_text_helper(index, w_head_end, clean_text, w_text)
    clean_text, _ = assemble_clean_text_helper(index, len(w_text), clean_text, w_text);
    return clean_text, w_head_start_offset, w_head_end_offset

In [ ]:
def offset(raw_str, clean_str, head_start, head_end):
    diff = 0
    quote = False
    for ind in range(len(clean_str)):
        temp = ind + diff
        # print(ind)
        if clean_str[ind] == ' ':
            if clean_str[ind+1] == '\"': 
                if quote == False:
                    quote = True
                else:
                    clean_str = clean_str[:ind] + '\" ' + clean_str[ind+2:]
                    quote = False
        while raw_str[temp] != clean_str[ind] and temp < len(raw_str):
            temp += 1
            diff += 1
            if head_start > ind:
                head_start -= 1
                head_end -= 1
            elif head_end > ind:
                head_end -= 1
    return [head_start, head_end]

## 1. when you hit a quote, check if the 'quote' boolean is false or true; if false, set it to true. if quote is true and character before the quote is a ' ' AND quote is true (meaning you hit the first quote already), then replace with '\" ' and set quote back to false.

do we want to preprocess all the quotes? Currently we are only switching the ones that are before the end of the anchor (head_end)

In [ ]:
raw_str = "All other countries which are seemingly \nrefraining from acquiring such weapons are doing so not so much \nto escape being branded as \"evil\" but rather because they have \nneither the knowledge nor the money to join the \"nuclear club\"."
clean_str = "All other countries which are seemingly refraining from acquiring such weapons are doing so not so much to escape being branded as \"evil \"but rather because they have neither the knowledge nor the money to join the \"nuclear club \"."
offset(raw_str, clean_str, 218, 232)

In [ ]:
clean_str[231]

In [ ]:
raw_str[218:232]

In [ ]:
clean_str[218:232]

In [ ]:
raw_str2 = "One can name more than a dozen countries which are \ncontemplating the acquisition of mass destruction weapons at one \nspeed or another."
clean_str2 = "One can name more than a dozen countries which are contemplating the acquisition of mass destruction weapons at one speed or another."

In [ ]:
offset(raw_str2, clean_str2, 47, 65)

In [ ]:
raw_str2[47:65]

In [ ]:
clean_str2[47:64]

In [ ]:
def main():
    

In [ ]:


if __name__ == '__main__':
    main()